In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
df=pd.read_csv("data.csv")

In [ ]:
df.head()

In [ ]:
df

In [ ]:
df.describe



In [ ]:
columns= df.columns[2:]
df[columns]=df[columns].fillna(0)

In [21]:
df=df.copy()
df = df.dropna(subset=columns, how='all')
df['meterreading']= df[columns].mean(axis=1)
df['voltageavg']=df['meterreading']
threshold =0.5
daily_diff = df[columns].diff(axis=1).fillna(0)

df['suddendrop'] = (daily_diff < -threshold * df[columns].shift(axis=1)).any(axis=1).astype(int)


In [23]:
x = df[['meterreading', 'suddendrop', 'voltageavg']]
y = df['FLAG']
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42, stratify=y
)
model=LogisticRegression()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=4))


Confusion Matrix:
 [[11610    17]
 [ 1071    14]]

Classification Report:
               precision    recall  f1-score   support

           0     0.9155    0.9985    0.9552     11627
           1     0.4516    0.0129    0.0251      1085

    accuracy                         0.9144     12712
   macro avg     0.6836    0.5057    0.4902     12712
weighted avg     0.8759    0.9144    0.8759     12712



In [24]:
#revising the parameter - including sudden spike and changing the model training from LogisticRegression to RandomForestClassifier to improve the confusion matrix and catch non linear trends


In [25]:
df = df.dropna(subset=columns, how='all')
meterreading=df[columns].mean(axis=1)
from sklearn.ensemble import RandomForestClassifier
rollingmean=df[columns].T.rolling(window=3,min_periods=1).mean().T
dailydiff=rollingmean.diff(axis=1).fillna(0)
threshold=0.5
suddenspike= (dailydiff > threshold*rollingmean.shift(axis=1)).any(axis=1).astype(int)
suddendrop= (dailydiff < -threshold*rollingmean.shift(axis=1)).any(axis=1).astype(int)
features = pd.concat([meterreading.rename('meter_reading'),
                      suddendrop.rename('sudden_drop'),
                      suddenspike.rename('sudden_spike')], axis=1)
df_final = pd.concat([df[['CONS_NO', 'FLAG']].reset_index(drop=True), features.reset_index(drop=True)], axis=1)

X = df_final[['meter_reading', 'sudden_drop', 'sudden_spike']]
y = df_final['FLAG']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
rf = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=4))



Confusion Matrix:
 [[10680   947]
 [  932   153]]

Classification Report:
               precision    recall  f1-score   support

           0     0.9197    0.9186    0.9191     11627
           1     0.1391    0.1410    0.1400      1085

    accuracy                         0.8522     12712
   macro avg     0.5294    0.5298    0.5296     12712
weighted avg     0.8531    0.8522    0.8526     12712



In [29]:
#improving the classweight- to increase the efficiency
rf = RandomForestClassifier(n_estimators=200, random_state=42, class_weight={0:1,1:50})

In [30]:
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=4))



Confusion Matrix:
 [[10678   949]
 [  932   153]]

Classification Report:
               precision    recall  f1-score   support

           0     0.9197    0.9184    0.9191     11627
           1     0.1388    0.1410    0.1399      1085

    accuracy                         0.8520     12712
   macro avg     0.5293    0.5297    0.5295     12712
weighted avg     0.8531    0.8520    0.8526     12712

